In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
from src.interface import TrainerModule
TrainerModule.main(stage = 0 , resume = 0 , checkname= 1)


24-05-23 23:29:25|MOD:display     |: Model Specifics:
24-05-23 23:29:25|MOD:display     |: Start Process [Data] at Thu May 23 23:29:25 2024!


Use device name: NVIDIA GeForce RTX 4090
--Process Queue : Data + Fit + Test
--Start Training New!
--Model_name is set to gru.2!
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gru.2',
 'model_module': 'lgbm',
 'model_data_type': 'day',
 'model_types': ['best'],
 'beg_date': 20170103,
 'end_date': 99991231,
 'sample_method': 'train_shuffle',
 'shuffle_option': 'epoch',
 'lgbm_ensembler': True}
{'hidden_dim': [32],
 'seqlens': [{'day': 30, '30m': 30, 'dms': 30}],
 'tra_seqlens': [{'hist_loss': 40}],
 'dropout': [0.1],
 'enc_in': [True],
 'enc_att': [False],
 'rnn_type': ['lstm'],
 'rnn_att': [False],
 'rnn_layers': [2],
 'dec_mlp_layers': [2],
 'num_output': [1],
 'kernel_size': [3],
 'hidden_as_factor': [False],
 'ordered_param_group': [False],
 'tra_num_states': [5],
 'resnet_blocks': [2]}
use /home/mengkjin/Workspace/learndl/data/DataSet/day.20240429.pt


24-05-23 23:29:28|MOD:display     |: Finish Process [Data], Cost 2.2 Secs
24-05-23 23:29:28|MOD:display     |: Start Process [Fit] at Thu May 23 23:29:28 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


24-05-23 23:29:39|MOD:display     |: gru.2 #0 @20170103|  |Train 0.0000 Valid 0.0000 BestVal 0.0000|Cost  0.2Min,  9.3Sec/Ep
24-05-23 23:29:51|MOD:display     |: gru.2 #0 @20170704|  |Train 0.0000 Valid 0.0000 BestVal 0.0000|Cost  0.1Min,  8.9Sec/Ep
24-05-23 23:30:03|MOD:display     |: gru.2 #0 @20171226|  |Train 0.0000 Valid 0.0000 BestVal 0.0000|Cost  0.2Min,  9.2Sec/Ep
24-05-23 23:30:17|MOD:display     |: gru.2 #0 @20180627|  |Train 0.0000 Valid 0.0000 BestVal 0.0000|Cost  0.2Min, 11.6Sec/Ep
24-05-23 23:30:29|MOD:display     |: gru.2 #0 @20181220|  |Train 0.0000 Valid 0.0000 BestVal 0.0000|Cost  0.2Min,  9.4Sec/Ep
24-05-23 23:30:41|MOD:display     |: gru.2 #0 @20190624|  |Train 0.0000 Valid 0.0000 BestVal 0.0000|Cost  0.2Min,  9.5Sec/Ep
24-05-23 23:30:54|MOD:display     |: gru.2 #0 @20191217|  |Train 0.0000 Valid 0.0000 BestVal 0.0000|Cost  0.2Min,  9.9Sec/Ep
24-05-23 23:31:07|MOD:display     |: gru.2 #0 @20200617|  |Train 0.0000 Valid 0.0000 BestVal 0.0000|Cost  0.2Min, 11.0Sec/Ep


score function of [spearman] calculated and success!


20170103     0.0346
20170704     0.0407
20171226     0.0642
20180627     0.0635
20181220     0.0781
20190624     0.0616
20191217     0.0506
20200617     0.0599
20201214     0.0461
20210615     0.0566
20211209     0.0515
20220613     0.0703
20221206     0.0587
20230606     0.0440
20231201     0.0802
AllTimeAvg   0.0567
AllTimeSum   101.42
Std          0.0727
TValue        32.98
AnnIR        3.8205
24-05-23 23:33:04|MOD:display     |: Finish Process [Test], Cost 16.8 Secs
24-05-23 23:33:04|MOD:time        |: Main Process Finished! Cost 3 Minutes 38.2 Seconds


In [ ]:
# prepare data
from src.interface import NetDataModule
NetDataModule.prepare_data()